# 🧠 RAG vs Non-RAG Evaluation with GPT-4o

This notebook evaluates GPT-4o performance on a synthetic longitudinal clinical dataset using both RAG and non-RAG approaches.

In [1]:
!pip install faiss-cpu sentence-transformers pandas openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 926.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

## 📁 Load the Clinical Notes Dataset

In [37]:
import pandas as pd
#df = pd.read_csv('/content/sample_data/synthetic_clinical_notes_1500.csv')
df = pd.read_csv('/content/sample_data/baby_ziya_1M_tokens_benchmark_ready.csv')
df.head()

,note_id,question_id,is_needle,note_type,author,phase,date,text
0,9c09354b-f50f-4516-aa23-4e3dbd8a93cb,Q1,True,Clinical Note,Nicu Specialist,nicu,2023-01-01,Physical exam shows significant hypotonia in a...
1,169db6d0-2e6b-415f-9efe-a5b05ae82575,Q2,True,Clinical Note,Nicu Specialist,nicu,2023-01-02,MRI confirms Dandy-Walker malformation as prev...
2,aa970905-3655-41df-bcd8-d1714a8c4e00,Q3,True,Clinical Note,Age_1_2 Specialist,age_1_2,2023-01-03,Developmental assessment identifies global dev...
3,61242120-51d2-47db-bdaf-b80708959989,Q4,True,Clinical Note,Nicu Specialist,nicu,2023-01-04,NICU MRI re-read shows no evidence of cerebell...
4,2c7a240a-06dc-4487-acb1-e93e21ecfddb,Q5,True,Clinical Note,Age_3_5 Specialist,age_3_5,2023-01-05,MRI at age 5 shows mild ventricular enlargemen...


## 🧠 Generate Embeddings using SentenceTransformers (can replace with OpenAI if needed)

In [38]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
texts = df['text'].tolist()
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/1864 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## 🔍 Build FAISS Vector Store

In [39]:
import faiss

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)
note_map = df[['note_id', 'note_type', 'date', 'text']].to_dict(orient='records')

## 🔎 Define Retrieval Function

In [40]:
def search_notes(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return [note_map[i] for i in indices[0]]

## ❓ Define Evaluation Questions

In [41]:
questions = [
"When was hypotonia first documented?",
"Was Dandy-Walker malformation ever reassessed after birth?",
"What is the earliest note of developmental delay?",
"Was there disagreement between fetal and NICU imaging?",
"Could the headaches at age 5 relate to prior findings?",
"Was seizure-like activity ever noted?",
"Was there a family history of migraines noted?",
"Has visual tracking difficulty been persistent over time?",
"Was there any concern of neglect or environment noted?",
"Was a neurology referral placed before age 1?"
]

## 🧪 Evaluate RAG vs Non-RAG using GPT-4o

In [42]:

import os
import time
import pandas as pd
from typing import List, Dict
import openai # Added import for the except block


os.environ['OPENAI_API_KEY'] = "sk-proj--"  # Replace with your actual API key


# Initialize OpenAI client
client = OpenAI()  # Assumes API key is set in environment variable OPENAI_API_KEY

results_eval: List[Dict] = []
non_rag_metrics = {"input_tokens": 0, "output_tokens": 0, "response_time": 0.0}
rag_metrics = {"input_tokens": 0, "output_tokens": 0, "retrieval_time": 0.0, "response_time": 0.0}

def safe_context_creation(texts: List[str], max_length: int = 50000) -> str:
    """Safely join text contexts while respecting token limits."""
    context = ""
    for text in texts:
        if len(context) + len(text) + 2 < max_length:
            context += text + "\n\n"
        else:
            break
    return context.strip()

try:
    for q in questions:
        if not isinstance(q, str) or not q.strip():
            print(f"Skipping invalid question: {q}")
            continue

        # Non-RAG Context
        #sample 50
        #full_context = safe_context_creation(df['text'].tolist()[:50])
        #all 1500
        full_context = safe_context_creation(df['text'].tolist())
        if not full_context:
            print(f"Warning: Empty non-RAG context for question: {q}")
            continue
        non_rag_prompt = f"Context:\n{full_context}\n\nQuestion:\n{q}\n\nAnswer:"

        # RAG Context
        start_time = time.time()
        top_notes = search_notes(q, top_k=5)
        rag_metrics["retrieval_time"] += time.time() - start_time
        if not top_notes:
            print(f"Warning: No notes retrieved for question: {q}")
            rag_context = ""
        else:
            rag_context = safe_context_creation([note['text'] for note in top_notes])
        rag_prompt = f"Context:\n{rag_context}\n\nQuestion:\n{q}\n\nAnswer:"

        try:
            # Non-RAG Generation
            start_time = time.time()
            non_rag_resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": non_rag_prompt}],
                temperature=0.3,
                max_tokens=1000
            )
            non_rag_metrics["response_time"] += time.time() - start_time
            non_rag_answer = non_rag_resp.choices[0].message.content
            non_rag_metrics["input_tokens"] += non_rag_resp.usage.prompt_tokens
            non_rag_metrics["output_tokens"] += non_rag_resp.usage.completion_tokens

            # RAG Generation
            start_time = time.time()
            rag_resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": rag_prompt}],
                temperature=0.3,
                max_tokens=1000
            )
            rag_metrics["response_time"] += time.time() - start_time
            rag_answer = rag_resp.choices[0].message.content
            rag_metrics["input_tokens"] += rag_resp.usage.prompt_tokens
            rag_metrics["output_tokens"] += rag_resp.usage.completion_tokens

            results_eval.append({
                "question": q,
                "non_rag_response": non_rag_answer,
                "rag_response": rag_answer
            })

        except openai.OpenAIError as e:
            print(f"API error for question '{q}': {str(e)}")
            continue
        except Exception as e:
            print(f"Unexpected error for question '{q}': {str(e)}")
            continue

    # --- This block was moved outside the for-loop ---

    # Print responses
    if not results_eval:
        print("No results to display.")
    else:
        for res in results_eval:
            print(f"\nQ: {res['question']}\n"
                  f"--- Non-RAG Answer ---\n{res['non_rag_response']}\n"
                  f"--- RAG Answer ---\n{res['rag_response']}")

    # Performance Summary
    print("\n📊 Performance Summary")
    input_cost_per_1k = 0.00015  # $0.15 per 1M tokens / 1000
    output_cost_per_1k = 0.0006 # $0.60 per 1M tokens / 1000

    non_rag_input_cost = non_rag_metrics["input_tokens"] / 1000 * input_cost_per_1k
    non_rag_output_cost = non_rag_metrics["output_tokens"] / 1000 * output_cost_per_1k
    non_rag_total_cost = non_rag_input_cost + non_rag_output_cost

    rag_input_cost = rag_metrics["input_tokens"] / 1000 * input_cost_per_1k
    rag_output_cost = rag_metrics["output_tokens"] / 1000 * output_cost_per_1k
    rag_total_cost = rag_input_cost + rag_output_cost

    print("\nNon-RAG Performance:")
    print(f"  Input Tokens: {non_rag_metrics['input_tokens']}")
    print(f"  Output Tokens: {non_rag_metrics['output_tokens']}")
    print(f"  Response Time: {non_rag_metrics['response_time']:.2f} seconds")
    print(f"  Estimated Cost: ${non_rag_total_cost:.4f}")

    print("\nRAG Performance:")
    print(f"  Input Tokens: {rag_metrics['input_tokens']}")
    print(f"  Output Tokens: {rag_metrics['output_tokens']}")
    print(f"  Retrieval Time: {rag_metrics['retrieval_time']:.2f} seconds")
    print(f"  Response Time: {rag_metrics['response_time']:.2f} seconds")
    print(f"  Total Time (Retrieval + Response): {rag_metrics['retrieval_time'] + rag_metrics['response_time']:.2f} seconds")
    print(f"  Estimated Cost: ${rag_total_cost:.4f}")

    # Total Metrics
    total_input_tokens = non_rag_metrics["input_tokens"] + rag_metrics["input_tokens"]
    total_output_tokens = non_rag_metrics["output_tokens"] + rag_metrics["output_tokens"]
    total_cost = non_rag_total_cost + rag_total_cost
    print(f"\n🧾 Total Input Tokens: {total_input_tokens}")
    print(f"🧾 Total Output Tokens: {total_output_tokens}")
    print(f"💸 Total Estimated Cost: ${total_cost:.4f}")

except Exception as e:
    print(f"Fatal error: {str(e)}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Q: When was hypotonia first documented?
--- Non-RAG Answer ---
Hypotonia was first documented during the physical exam, as indicated in the provided context.
--- RAG Answer ---
Hypotonia was first documented during the physical exam.

Q: Was Dandy-Walker malformation ever reassessed after birth?
--- Non-RAG Answer ---
Yes, the Dandy-Walker malformation was reassessed after birth. The MRI re-read during the NICU stay showed no evidence of cerebellar abnormality, which contradicted the fetal findings of cerebellar hypoplasia and the posterior fossa cyst consistent with the Dandy-Walker spectrum. This indicates that the condition was evaluated again after birth, leading to differing conclusions from the initial fetal imaging.
--- RAG Answer ---
The provided context does not indicate whether Dandy-Walker malformation was reassessed after birth. It only mentions that MRI and ultrasound confirmed the diagnosis as noted in fetal imaging. Additional information would be needed to determine if